In [484]:
import os, os.path
import numpy as np
import pandas as pd
import data_structures as ds
import setup_analysis as sa
import support_functions as sf
import importlib
import time
import warnings
import matplotlib.pyplot as plt
import model_afolu as ma
import model_circular_economy as mc
import model_ippu as mi
import model_socioeconomic as ms

importlib.reload(ds)
importlib.reload(sa)
importlib.reload(sf)
importlib.reload(ma)
importlib.reload(mc)
importlib.reload(mi)
importlib.reload(ms)

<module 'model_socioeconomic' from '/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_socioeconomic.py'>

In [391]:
sa.model_attributes.configuration.get("historical_harvested_wood_products_method")

'back_project'

In [230]:
#[x for x in df_tmp["Item"] if ("pulp" in x.lower()) or ("paper" in x.lower())
df_tmp2[
    df_tmp["Area"].isin(["Brazil"]) &
    df_tmp["Year"].isin([2019]) & 
    df_tmp["Unit"].isin(["m3"]) &
    df_tmp["Element"].isin(["Production"]) & 
    df_tmp["Item"].isin(items)
]
#142989000 - 78749000

,Area Code,Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag,Item_weight,sum_val
187135,21,Brazil,1865,Industrial roundwood,5516,Production,2019,2019,m3,142989000,Im,1,142989000
188562,21,Brazil,2038,"Pulpwood, round and split, all species (produc...",5516,Production,2019,2019,m3,78749000,Im,-1,-78749000


In [303]:
df_tmp = pd.read_csv("/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/ingestion/FAOSTAT/Forestry_E_All_Data_(Normalized)/Forestry_E_All_Data_(Normalized)_utf8.csv")

yrs = range(2011, 2020)

dict_add = {"Wood fuel": 0.5, "Industrial roundwood": 1, "Pulpwood, round and split, all species (production)": -1}
items = list(dict_add.keys())

df_tmp2 = df_tmp[
    #df_tmp["Area"].isin(["Mexico"]) &
    df_tmp["Year"].isin(yrs) & 
    df_tmp["Unit"].isin(["m3"]) &
    df_tmp["Element"].isin(["Production"]) & 
    df_tmp["Item"].isin(items)
].copy()
df_tmp2["Item_weight"] = df_tmp2["Item"].replace(dict_add)
df_tmp2["sum_val"] = np.array(df_tmp2["Item_weight"])*np.array(df_tmp2["Value"])

scalar_m3_to_tonnes = 0.5

fields_group = ["Area", "Year"]
fields_agg = ["sum_val"]
dict_agg = dict(zip(fields_group, ["first" for x in fields_group]))
dict_agg.update(dict(zip(fields_agg, ["sum" for x in fields_agg])))
df_tmp3 = df_tmp2.groupby(fields_group).agg(dict_agg).reset_index(drop = True)
field_new = "prodinit_ippu_wood_tonne"
df_tmp3.rename(columns = {"sum_val": field_new}, inplace = True)
df_tmp3[field_new] = np.array(df_tmp3[field_new])*scalar_m3_to_tonnes


df_persons = df_tmp[
    #df_tmp["Area"].isin(["Mexico"]) &
    df_tmp["Year"].isin(yrs) & 
    df_tmp["Unit"].isin(["m3"]) &
    df_tmp["Element"].isin(["Production"]) & 
    df_tmp["Item"].isin(["Wood fuel"])
].copy()



field_new_2 = "deminit_frst_wood_home_use_kg_per_hh_NOTEDIVIDEBYPOPULATIONFIRST"
df_persons[field_new_2] = np.array(df_persons["Value"]*(1 - dict_add["Wood fuel"]))*scalar_m3_to_tonnes*1000


df_out = pd.merge(df_tmp3, df_persons[fields_group + [field_new_2]])
           
#df_tmp3[field_new] = np.array(df_tmp3[field_new])*0.5

#].to_csv("/Users/jsyme/Desktop/tmp.csv")

In [309]:
#df_afolu_trajectories[[x for x in df_afolu_trajectories if "popul" in x]]



In [453]:
df_out[df_out["Area"] == "Costa Rica"]
#df_tmp2[df_tmp2["Area"].isin(["Mexico"]) & df_tmp2["Year"].isin([2019])]["sum_val"].sum()*scalar_m3_to_tonnes

#df_persons[df_persons["Area"].isin(["Mexico"]) & df_persons["Year"].isin([2019])][""]
814419250.0/1541331

528.3869915027985

In [488]:
# df_project_afolu[[x for x in df_project_afolu.columns if "emission_co2e_subsector_total" in x]]
importlib.reload(mi)
#importlib.reload(me)
importlib.reload(ma)
df_afolu_trajectories0 = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_complete.csv"))
# test
df_afolu_trajectories = pd.read_csv("/Users/jsyme/Downloads/data_complete_future_2022_07_19.csv")
df_afolu_trajectories["frac_agrc_agriculture_production_lost"] = 0.21 # temp
df_afolu_trajectories = df_afolu_trajectories[df_afolu_trajectories["Nation"] == "argentina"].reset_index(drop = True)
model_afolu = ma.AFOLU(sa.model_attributes)
model_circ = mc.CircularEconomy(sa.model_attributes)
model_ippu = mi.IPPU(sa.model_attributes)
#warnings.filterwarnings("ignore")
t0 = time.time()#
#df_afolu_trajectories["timespan_ippu_average_lifetime_of_housing"] = 70
#df_afolu_trajectories["dem_ippu_average_materials_tonne_per_household_wood"] = [0.1, 0.2, 0.3, 0.4, 0.9, 1.2] + [1.2 for x in range(30)]
#df_afolu_trajectories["dem_ippu_average_materials_tonne_per_household_cement"] = np.array([1, 0.9, 0.8, 0.7] + [0 for x in range(32)]) 
#df_afolu_trajectories["prodinit_ippu_cement_tonne"] = 10
#df_afolu_trajectories["frac_scoe_heat_energy_residential_solid_biomass"] = [0.13, 0.12,0.11,0.1, 0.09, 0.05, 0.01] + [0.01 for x in range(29) ]
df_project_afolu = model_afolu.project(df_afolu_trajectories)
t1 = time.time()
#print(t1 - t0)
#df_project_circ = model_circ.project(df_afolu_trajectories)
#df_ippu_in = sf.merge_output_df_list([df_afolu_trajectories, df_project_circ], sa.model_attributes, "concatenate")

#df_project_ippu = model_ippu.project(df_ippu_in)
#sa.model_attributes.get_standard_variables(df_project_ippu, model_ippu.modvar_ippu_qty_total_production)
#df_project_afolu[[x for x in df_project_afolu if "emission_co2e_n2o_soil" in x]]#/.sum(axis = 1)
df_project_afolu[[x for x in df_project_afolu if "frst" in x]]

#print(df_project_afolu["emission_co2e_co2_frst_harvested_wood_products"])
df_project_afolu[[x for x in df_project_afolu if "qty_agrc" in x and "lost" in x]]


,qty_agrc_food_produced_lost_tonne
0,3.840565e+07
1,3.832212e+07
2,3.823404e+07
3,3.814134e+07
4,3.804396e+07
5,3.794183e+07
6,3.825365e+07
7,3.856529e+07
8,3.887675e+07
9,3.918803e+07


In [451]:
df_afolu_trajectories

,Nation,time_period,population_gnrl_urban,population_gnrl_rural,gdp_mmm_usd,area_gnrl_country_ha,avgmass_lvst_animal_buffalo_kg,avgmass_lvst_animal_cattle_dairy_kg,avgmass_lvst_animal_cattle_nondairy_kg,avgmass_lvst_animal_chickens_kg,...,vol_ippu_recycled_metals_m3_ww_per_tonne_production,vol_ippu_recycled_paper_m3_ww_per_tonne_production,vol_ippu_recycled_plastic_m3_ww_per_tonne_production,vol_ippu_recycled_rubber_and_leather_m3_ww_per_tonne_production,vol_ippu_recycled_textiles_m3_ww_per_tonne_production,vol_ippu_rubber_and_leather_m3_ww_per_tonne_production,vol_ippu_textiles_m3_ww_per_tonne_production,vol_ippu_wood_m3_ww_per_tonne_production,frac_agrc_agriculture_production_wasted,population_gnrl_total
0,argentina,0,38471261.51,3573833.037,659.042597,278040000,317.308432,456.051264,325.770509,1.146395,...,3.4,3,4,16,16,25,25,25,0.292,4.204509e+07
1,argentina,1,38793803.69,3556178.993,649.225462,278040000,317.308432,456.051264,325.770509,1.146395,...,3.4,3,4,16,16,25,25,25,0.292,4.234998e+07
2,argentina,2,39116345.86,3538524.948,639.408328,278040000,317.308432,456.051264,325.770509,1.146395,...,3.4,3,4,16,16,25,25,25,0.292,4.265487e+07
3,argentina,3,39438888.04,3520870.904,629.591193,278040000,317.308432,456.051264,325.770509,1.146395,...,3.4,3,4,16,16,25,25,25,0.292,4.295976e+07
4,argentina,4,39761430.21,3503216.860,619.774059,278040000,317.308432,456.051264,325.770509,1.146395,...,3.4,3,4,16,16,25,25,25,0.292,4.326465e+07
5,argentina,5,40083972.38,3485562.816,609.956924,278040000,317.308432,456.051264,325.770509,1.146395,...,3.4,3,4,16,16,25,25,25,0.292,4.356954e+07
6,argentina,6,40386364.08,3457040.991,616.564095,278040000,317.308432,456.051264,325.770509,1.146395,...,3.4,3,4,16,16,25,25,25,0.292,4.384341e+07
7,argentina,7,40688755.79,3428519.166,623.171265,278040000,317.308432,456.051264,325.770509,1.146395,...,3.4,3,4,16,16,25,25,25,0.292,4.411727e+07
8,argentina,8,40991147.49,3399997.341,629.778436,278040000,317.308432,456.051264,325.770509,1.146395,...,3.4,3,4,16,16,25,25,25,0.292,4.439114e+07
9,argentina,9,41293539.19,3371475.517,636.385606,278040000,317.308432,456.051264,325.770509,1.146395,...,3.4,3,4,16,16,25,25,25,0.292,4.466501e+07


In [290]:

df_afolu_trajectories["timespan_ippu_average_lifetime_of_housing"] = 10
df_afolu_trajectories["dem_ippu_average_materials_tonne_per_household_wood"] = [0.1, 0.2, 0.3, 0.4, 0.9, 1.2] + [1.2 for x in range(30)]
#df_afolu_trajectories["dem_ippu_average_materials_tonne_per_household_cement"] = np.array([1, 0.9, 0.8, 0.7] + [0 for x in range(32)]) 
#df_afolu_trajectories["prodinit_ippu_cement_tonne"] = 10
#df_afolu_trajectories["frac_scoe_heat_energy_residential_solid_biomass"] = [0.13, 0.12,0.11,0.1, 0.09, 0.05, 0.01] + [0.01 for x in range(29) ]
df_project_afolu = model_afolu.project(df_afolu_trajectories)

In [361]:
#sa.model_attributes.build_varlist("IPPU")#, model_ippu.modvar_ippu_max_recycled_material_ratio)
#df_afolu_trajectories.shape


In [375]:
model_ippu = mi.IPPU(sa.model_attributes)
varm = model_ippu.modvar_ippu_max_recycled_material_ratio

arr1 = sa.model_attributes.get_standard_variables(df_afolu_trajectories, varm, return_type = "array_base", expand_to_all_cats = True)
arr1[:, attr_ippu.get_key_value_index("paper")]
#sa.model_attributes.get_scalar("Initial Non-Energy Fuel Use", "mass")
#sa.model_attributes.get_variable_characteristic("Initial Non-Energy Fuel Use", "$UNIT-MASS$")

array([0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6,
       0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6,
       0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6])

In [377]:
importlib.reload(ds)
importlib.reload(ma)

model_afolu = ma.AFOLU(sa.model_attributes)
#model_circ = mc.CircularEconomy(sa.model_attributes)
#model_ippu = mi.IPPU(sa.model_attributes)
warnings.filterwarnings("ignore")
df_project_afolu = model_afolu.project(df_afolu_trajectories)

df_project_afolu[[x for x in df_project_afolu if "emission_co2e_n2o_soil" in x]].sum(axis = 1)
df_project_afolu[[x for x in df_project_afolu if "frst" in x and "emission" in x]]#.sum(axis = 1)

[3.58662904 3.64319854 3.70066026 3.7590283  3.81831693 3.87854068
 3.93971431 4.00185278 4.06497132 4.12908539 4.19421069 4.27868884
 4.35010397 4.40912514 4.4730651  4.53550754 4.60186845 4.67053456
 4.74379887 4.8214971  4.90001911 4.97937974 5.05958249 5.14063644
 5.22254875 5.30532983 5.388989   5.4735475  5.55901905 5.64543633
 5.73281802 5.82118139 5.91054204 6.00091228 6.09231211 6.18475895
 6.27826589 6.37285301 6.46852925 6.56529557 6.66316441 6.76215277
 6.8622802  6.96356842 7.06603641 7.16970685]


,emission_co2e_co2_frst_sequestration_mangroves,emission_co2e_co2_frst_sequestration_primary,emission_co2e_co2_frst_sequestration_secondary,emission_co2e_ch4_frst_methane_mangroves,emission_co2e_ch4_frst_methane_primary,emission_co2e_ch4_frst_methane_secondary,emission_co2e_co2_frst_forest_fires,emission_co2e_co2_frst_harvested_wood_products,emission_co2e_subsector_total_frst
0,-0.098780,-5.237938,-1.105878,0.001312,0.060891,0.025712,0.000180,-0.406721,-6.761223
1,-0.098629,-5.223194,-1.093686,0.001310,0.060720,0.025428,0.000178,-0.462721,-6.790593
2,-0.098375,-5.208907,-1.078163,0.001307,0.060554,0.025067,0.000176,-0.537133,-6.835473
3,-0.098081,-5.195096,-1.061796,0.001303,0.060393,0.024687,0.000174,-0.596382,-6.864798
4,-0.097764,-5.181750,-1.045307,0.001299,0.060238,0.024303,0.000172,-0.641623,-6.880432
5,-0.097441,-5.168867,-1.029302,0.001295,0.060088,0.023931,0.000170,-0.690918,-6.901045
6,-0.097092,-5.156436,-1.012820,0.001290,0.059944,0.023548,0.000167,-0.737577,-6.918976
7,-0.096713,-5.144451,-0.995659,0.001285,0.059804,0.023149,0.000165,-0.787183,-6.939603
8,-0.096293,-5.132905,-0.977297,0.001279,0.059670,0.022722,0.000162,-0.837997,-6.960659
9,-0.095844,-5.121791,-0.958192,0.001273,0.059541,0.022278,0.000160,-0.892318,-6.984893


In [ ]:
df

In [65]:
df_afolu_trajectories = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_complete.csv"))
df_afolu_trajectories.shape                      
#self.model_attributes.get_standard_variables(df_afolu_trajectories, self.modvar_lvst_animal_weight, True, "array_base", expand_to_all_cats = True)                        

(36, 1578)

In [411]:
#vars_afolu = sa.model_attributes.build_varlist("Soil Management", variable_type = "input")
vars_afolu = sa.model_attributes.build_varlist("Agriculture", variable_type = "input")
#vars_afolu += sa.model_attributes.build_varlist("Forest", variable_type = "input")
#vars_afolu += sa.model_attributes.build_varlist("Land Use", variable_type = "input")
vars_afolu = [x for x in vars_afolu if (x not in df_afolu_trajectories.columns)]
vars_afolu.sort()

for k in vars_afolu:
    print(k)
    

frac_agrc_agriculture_production_wasted


In [409]:
print(model_afolu.modvar_lsmm_emissions_n2o)

:math:\text{N}_2\text{O} Emissions from Manure Management


In [80]:

#df_project_afolu = model_afolu.project(df_afolu_trajectories)
#df_cs_integrated = sf.merge_output_df_list([df_cs_integrated, df_project_afolu], sa.model_attributes, "concatenate")





In [16]:
dict_arrs_lsmm_frac_manure = sa.model_attributes.get_multivariables_with_bounded_sum_by_category(
    df_afolu_trajectories,
    model_afolu.modvar_lvst_list_mm_fractions,
    1,
    force_sum_equality = True,
    msg_append = "Energy fractions by category do not sum to 1. See definition of dict_arrs_inen_frac_energy."
)

In [26]:
sa.model_attributes.get_standard_variables(df_afolu_trajectories, model_afolu.modvar_lvst_genfactor_volatile_solids)

,genfactor_lvst_daily_volatile_solids_buffalo,genfactor_lvst_daily_volatile_solids_cattle_dairy,genfactor_lvst_daily_volatile_solids_cattle_nondairy,genfactor_lvst_daily_volatile_solids_chickens,genfactor_lvst_daily_volatile_solids_goats,genfactor_lvst_daily_volatile_solids_horses,genfactor_lvst_daily_volatile_solids_mules,genfactor_lvst_daily_volatile_solids_pigs,genfactor_lvst_daily_volatile_solids_sheep
0,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
1,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
2,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
3,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
4,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
5,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
6,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
7,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
8,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
9,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083


In [412]:
df_emissions = pd.read_csv("/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/ingestion/FAOSTAT/Emissions_Totals_E_All_Data_(Normalized)/Emissions_Totals_E_All_Data_(Normalized).csv")



In [424]:
ai = set(df_emissions["Item"])
ai - all_items

{'Drained organic soils',
 'Drained organic soils (CO2)',
 'Drained organic soils (N2O)'}

In [436]:
all_items = set(df_emissions[
    df_emissions["Year"].isin([2019]) &
    df_emissions["Area"].isin(["Mexico"])
]["Item"])

all_elems = set([x for x in list(df_emissions["Element"]) if "Emissions (CO2eq) (AR5)" in x])

df_emissions[
    df_emissions["Year"].isin([2019]) &
    df_emissions["Area"].isin(["Mexico"]) & 
    df_emissions["Element"].isin(all_elems)
]

,Area Code,Area,Item Code,Item,Element Code,Element,Year Code,Year,Source Code,Source,Unit,Value,Flag,Note
588739,138,Mexico,5058,Enteric Fermentation,723113,Emissions (CO2eq) (AR5),2019,2019,3050,FAO TIER 1,kilotonnes,64402.7130,Fc,NaN
589164,138,Mexico,5059,Manure Management,723113,Emissions (CO2eq) (AR5),2019,2019,3050,FAO TIER 1,kilotonnes,4123.7593,Fc,NaN
589347,138,Mexico,5060,Rice Cultivation,723113,Emissions (CO2eq) (AR5),2019,2019,3050,FAO TIER 1,kilotonnes,123.8123,Fc,NaN
589652,138,Mexico,5061,Synthetic Fertilizers,723113,Emissions (CO2eq) (AR5),2019,2019,3050,FAO TIER 1,kilotonnes,7870.1684,Fc,NaN
589957,138,Mexico,5062,Manure applied to Soils,723113,Emissions (CO2eq) (AR5),2019,2019,3050,FAO TIER 1,kilotonnes,3041.1180,Fc,NaN
590262,138,Mexico,5063,Manure left on Pasture,723113,Emissions (CO2eq) (AR5),2019,2019,3050,FAO TIER 1,kilotonnes,18578.4776,Fc,NaN
590567,138,Mexico,5064,Crop Residues,723113,Emissions (CO2eq) (AR5),2019,2019,3050,FAO TIER 1,kilotonnes,1795.7837,Fc,NaN
590987,138,Mexico,5066,Burning - Crop residues,723113,Emissions (CO2eq) (AR5),2019,2019,3050,FAO TIER 1,kilotonnes,702.7018,Fc,NaN
591049,138,Mexico,6750,Net Forest conversion,723113,Emissions (CO2eq) (AR5),2019,2019,3050,FAO TIER 1,kilotonnes,14821.3956,Fc,NaN
591109,138,Mexico,6751,Forestland,723113,Emissions (CO2eq) (AR5),2019,2019,3050,FAO TIER 1,kilotonnes,-373.6163,Fc,NaN


In [419]:
df_tmp = pd.read_csv("/Users/jsyme/Downloads/FAOSTAT_data_7-19-2022 (1).csv")


In [432]:
#df_tmp
all_elems

{'Emissions (CO2eq) (AR5)',
 'Emissions (CO2eq) from CH4 (AR5)',
 'Emissions (CO2eq) from N2O (AR5)'}

In [430]:
list(df_emissions["Element"])

['Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
